In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Read the CSV file into a DataFrame
df = pd.read_csv('data.csv')

# Create a new DataFrame considering only 'timestamp' and 'valueTfuel'
considered_df = df[['timestamp', 'valueTfuel']].copy()

In [5]:
considered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   100 non-null    int64  
 1   valueTfuel  86 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.7 KB


In [16]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

def predict_value_after_months(data_csv, n_months_ahead):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(data_csv)

    # Create a new DataFrame considering only 'timestamp' and 'valueTfuel'
    considered_df = df[['timestamp', 'valueTfuel']].copy()

    # Remove rows with null values in the considered DataFrame
    considered_df.dropna(inplace=True)

    # Perform log transformation on 'valueTfuel'
    considered_df['log_valueTfuel'] = np.log1p(considered_df['valueTfuel'])

    # Remove rows with log_Tfuel = -inf
    considered_df = considered_df[considered_df['log_valueTfuel'] != -np.inf]

    # Create a new DataFrame with 'timestamp' and 'log_valueTfuel'
    final_df = considered_df[['timestamp', 'log_valueTfuel']].copy()

    # Convert the 'timestamp' column to datetime type
    final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

    # Sort the DataFrame by 'timestamp'
    final_df.sort_values('timestamp', inplace=True)

    # Set 'timestamp' as the index of the DataFrame
    final_df.set_index('timestamp', inplace=True)

    # Define a function to create a supervised learning dataset
    def create_dataset(data, n_features):
        X, y = [], []
        for i in range(len(data)-n_features-1):
            X.append(data[i:(i+n_features), 0])
            y.append(data[i + n_features, 0])
        return np.array(X), np.array(y)

    # Define the number of previous time steps to consider
    n_steps = 30

    # Create the supervised learning dataset
    X, y = create_dataset(final_df.values, n_steps)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Reshape the input data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Define the RNN model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(n_steps, 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32)

    # Predict the log_valueTfuel after n_months_ahead
    last_month_data = final_df.tail(n_steps).values
    last_month_data = np.reshape(last_month_data, (1, n_steps, 1))

    for _ in range(n_months_ahead):
        predicted_value = model.predict(last_month_data)
        last_month_data = np.append(last_month_data, np.expand_dims(predicted_value, axis=1), axis=1)
        last_month_data = last_month_data[:, -n_steps:, :]

    # Take anti-log to get the predicted value
    predicted_value = np.expm1(predicted_value)

    return predicted_value[0][0]




In [19]:
# Example usage
data_csv = 'data.csv'
n_months_ahead = 1
predicted_value = predict_value_after_months(data_csv, n_months_ahead)
print("Predicted Value after", n_months_ahead, "month(s):", predicted_value)

Epoch 1/50
2/2 [==============================] - 6s 42ms/step - loss: 54.1749
Epoch 2/50
2/2 [==============================] - 0s 33ms/step - loss: 44.7526
Epoch 3/50
2/2 [==============================] - 0s 35ms/step - loss: 35.8071
Epoch 4/50
2/2 [==============================] - 0s 34ms/step - loss: 28.3265
Epoch 5/50
2/2 [==============================] - 0s 33ms/step - loss: 22.4841
Epoch 6/50
2/2 [==============================] - 0s 34ms/step - loss: 18.1221
Epoch 7/50
2/2 [==============================] - 0s 33ms/step - loss: 14.7247
Epoch 8/50
2/2 [==============================] - 0s 34ms/step - loss: 12.1933
Epoch 9/50
2/2 [==============================] - 0s 34ms/step - loss: 9.9439
Epoch 10/50
2/2 [==============================] - 0s 35ms/step - loss: 8.2983
Epoch 11/50
2/2 [==============================] - 0s 36ms/step - loss: 6.9232
Epoch 12/50
2/2 [==============================] - 0s 34ms/step - loss: 5.7646
Epoch 13/50
2/2 [==============================] - 0s